In [1]:
# --- CELL 1: INSTALL & IMPORT ---
# Jalankan ini dulu sekali saja
try:
    import gradio
except ImportError:
    %pip install gradio transformers torch

import gradio as gr
from transformers import pipeline

# --- CELL 2: LOAD MODEL & JALANKAN WEB ---
# Pastikan folder ini ada di sebelah kiri (sejajar dengan file notebook ini)
model_path = "./model_distilbert_light_final"

print(f"📂 Sedang memuat model dari: {model_path}...")

# Load Pipeline (Otomatis baca model & tokenizer dari folder)
try:
    ner_pipeline = pipeline(
        "token-classification", 
        model=model_path, 
        tokenizer=model_path, 
        aggregation_strategy="simple" 
    )
    print("✅ Model berhasil dimuat!")
except OSError:
    print("❌ Error: Folder model tidak ditemukan. Pastikan path-nya benar.")

# Fungsi Logika (Backend)
def deteksi_medis(teks):
    hasil = ner_pipeline(teks)
    formatted_results = []
    cursor = 0
    for entitas in hasil:
        start, end = entitas['start'], entitas['end']
        label, score = entitas['entity_group'], entitas['score']
        
        if start > cursor:
            formatted_results.append((teks[cursor:start], None))
        
        if score > 0.40: # Tampilkan jika yakin > 40%
            formatted_results.append((teks[start:end], label))
        else:
            formatted_results.append((teks[start:end], None))    
        cursor = end
    
    if cursor < len(teks):
        formatted_results.append((teks[cursor:], None))
    return formatted_results

# Tampilan Web (Frontend)
custom_css = ".gradio-container {background-color: #f0f4f8}"

with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🏥 Medical Entity Recognition (DistilBERT Demo)
        **Status:** Ready for Deployment | **Model:** Light Version
        """
    )
    
    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="Catatan Medis", 
                placeholder="Paste text here...", 
                lines=4,
                value="The patient was prescribed Aspirin for severe hypertension and heart failure."
            )
            btn = gr.Button("Analisis Sekarang", variant="primary")
        
        with gr.Column():
            output_display = gr.HighlightedText(
                label="Hasil Deteksi",
                combine_adjacent=True,
                show_legend=True,
                color_map={"Chemical": "blue", "Disease": "red"}
            )

    btn.click(fn=deteksi_medis, inputs=input_text, outputs=output_display)

# Jalankan
print("🚀 Membuka aplikasi...")
demo.launch(share=True)

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.12.4-py3-none-any.whl.metadata (89 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached starlette-0.50.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached annotated_doc-0.0.4-py3-none-any.whl.metadata (6.6 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/23.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/23.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/23.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/23.0 M

c:\Users\macin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


📂 Sedang memuat model dari: ./model_distilbert_light_final...
✅ Model berhasil dimuat!
🚀 Membuka aplikasi...


C:\Users\macin\AppData\Local\Temp\ipykernel_2120\4224106239.py:54: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://7473d6e2fa9bcee01f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


c:\Users\macin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:392: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
